# Data Cleansing for Spotify Dataset

In [1]:
#sc.stop()
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
#conf = pyspark.SparkConf().setAppName('appName').setMaster('spark://192.168.11.128:8080')

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [2]:
df_weeklyTop200 = spark.read.csv('../datasets/spotify_top_200_weekly.csv', header=True)
df_track_details = spark.read.csv('../datasets/spotify_track_details.csv', header=True)
df_artists = spark.read.csv('../datasets/kaggle_artists.csv', header=True)

In [ ]:
df_artists.count()

In [ ]:
df_weeklyTop200.show()

In [ ]:
df_weeklyTop200.info()

In [ ]:
#df_weeklyTop200['Start_Date'] = df_weeklyTop200.apply(lambda row: row.date[:10],row.date[10:] axis = 1) 
df_weeklyTop200.describe(include='all')
#df_weeklyTop200.describe()



In [ ]:
df_weeklyTop200['date']

In [ ]:
#import pyspark.sql.functions as F

In [3]:
from pyspark.sql.functions import col, split
df_weeklyTop200 = df_weeklyTop200.withColumn("start_date", split(col("date"), "--").getItem(0)).withColumn("end_date", split(col("date"), "--").getItem(1))

df_weeklyTop200.show(2)
#df_weeklyTop200[['start_date', 'end_date']] = df_weeklyTop200.date #.str.split('--', expand=True)
#df_weeklyTop200

+--------+--------------------+-------------+-------+--------------------+------+--------------------+----------+----------+
|Position|          Track Name|       Artist|Streams|                date|region|          spotify_id|start_date|  end_date|
+--------+--------------------+-------------+-------+--------------------+------+--------------------+----------+----------+
|       1|   Someone You Loved|Lewis Capaldi|3338234|2019-03-01--2019-...|    gb|2TIlqbIneP0ZY1O0E...|2019-03-01|2019-03-08|
|       2|break up with you...|Ariana Grande|3041558|2019-03-01--2019-...|    gb|4kV4N9D1iKVxx1KLv...|2019-03-01|2019-03-08|
+--------+--------------------+-------------+-------+--------------------+------+--------------------+----------+----------+
only showing top 2 rows



In [ ]:
#df_weeklyTop200.distinct().show()
df_weeklyTop200.select('Artist').distinct().count()

In [ ]:
df_weeklyTop200.select('*').describe()

## Clean up the track_details data
### Lookup missing mb_id in the in the kaggle dataset

In [5]:
#get all rows with no mb_id

df_null_mbid = df_track_details.filter(df_track_details.artist_mbid.isNull())
df_null_mbid.count()


629

In [6]:
#check if any row has no artist name
df_null_mbid.filter(df_null_mbid.artist_name.isNull()).count()


0

In [16]:
#get unique rows with artist_name to avoid expensive join
df_unique_artist = df_null_mbid.dropDuplicates(['artist_name'])
df_unique_artist.count()

275

In [8]:
from pyspark.sql.functions import levenshtein 

In [ ]:
newDF=df_unique_artist.join(df_artists,levenshtein(df_unique_artist['artist_name'], df_artists['artist_mb']) < 1, 'leftouter')

In [ ]:
newDF.count()

In [ ]:
#print(newDF.count())
newDF[['artist_name','mbid', 'artist_mb']].show()

In [ ]:
#from fuzzywuzzy import fuzz

In [ ]:
#fuzz.token_sort_ratio(Str1,Str2)
#newDF=df_unique_artist.join(df_artists,fuzz.token_sort_ratio(df_unique_artist['artist_name'], df_artists['artist_mb']) > 90, 'left_semi')

In [9]:
df_musicbrainz_artist = spark.read.csv('../datasets/musicBrainz_ActiveArtists.csv', header=True)

In [10]:
df_musicbrainz_artist.show(2)

+--------------------+-----------+------------+-----+
|                 gid|       name|   sort_name|ended|
+--------------------+-----------+------------+-----+
|8972b1c1-6482-475...|      WIK▲N|       WIK▲N|False|
|49add228-eac5-4de...|Pete Moutso|Moutso, Pete|False|
+--------------------+-----------+------------+-----+
only showing top 2 rows



In [17]:
#get unique rows with artist_name to avoid expensive join
df_unique_musicbrainz_artist = df_musicbrainz_artist.dropDuplicates(['sort_name'])
df_unique_musicbrainz_artist.count()


1394800

In [24]:
newDF=df_unique_artist.join(df_unique_musicbrainz_artist,levenshtein(df_unique_artist['artist_name'], df_unique_musicbrainz_artist['sort_name']) < 2, 'left')

In [22]:
newDF.count()

275

In [25]:
newDF.filter(newDF.gid.isNull()).count()

84

In [21]:
newDF[['artist_name','gid', 'sort_name']].show()

+--------------------+--------------------+--------------------+
|         artist_name|                 gid|           sort_name|
+--------------------+--------------------+--------------------+
|              Brando|d5620c9c-220e-477...|              Brando|
|               Cadet|c872f796-1497-49d...|               Cadet|
|           CamelPhat|61314ff2-7b52-429...|           CamelPhat|
|      D-Block Europe|3325cdfa-8efc-466...|      D-Block Europe|
|                Deno|eb6af86f-4533-419...|                Deno|
|               Fredo|de67fa7b-9910-4a5...|               Fredo|
|               Giggs|c2beec92-0560-4fd...|               Giggs|
|         Loud Luxury|4c445344-9df2-407...|         Loud Luxury|
|                 NSG|ecebea6a-c064-440...|                 NSG|
|               Not3s|ef572aef-6d79-493...|               Not3s|
|           Sheck Wes|07ab04d4-bdbc-4c9...|           Sheck Wes|
|              Swarmz|59466d51-6806-41d...|              Swarmz|
|               Tainy|7e9